In [12]:
import pandas as pd

# train data
train_df = pd.read_csv("/Users/yahavbar/Documents/METL-Rosetta/METL-Rosetta/output/train_subset.tsv", sep="\t")
test_df = pd.read_csv("/Users/yahavbar/Documents/METL-Rosetta/METL-Rosetta/output/test_subset.tsv", sep="\t")
val_df = pd.read_csv("/Users/yahavbar/Documents/METL-Rosetta/METL-Rosetta/output/val_subset.tsv", sep="\t")

## Predefined feature models:

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

target_col = "total_score"

models_features = {
    "model_energy_only": [
        "fa_atr", "fa_rep", "fa_sol", "fa_elec", "fa_dun"
    ],
    "model_hbond_env": [
        "hbond_bb_sc", "hbond_lr_bb", "hbond_sc", "env", "vdw"
    ],
    "model_structural": [
        "buried_all", "buried_np", "contact_all",
        "total_sasa", "rg", "pack"
    ],
    "model_all_selected": [
        "fa_atr", "fa_rep", "fa_sol", "fa_elec", "fa_dun",
        "hbond_bb_sc", "hbond_lr_bb", "hbond_sc",
        "buried_all", "buried_np", "contact_all",
        "total_sasa", "rg", "pack"
    ],
    "model_energy_and_structural": [
        "fa_atr", "fa_rep", "fa_sol", "fa_elec", "fa_dun",
        "buried_all", "buried_np", "contact_all", "total_sasa",
        "rg", "pack"
    ]
}
results = {}
for name, feats in models_features.items():
    X_train = train_df[feats]
    y_train = train_df[target_col]

    X_val = val_df[feats]
    y_val = val_df[target_col]

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)

    results[name] = {
        "features": feats,
        "mse_val": mse,
        "r2_val": r2,
        "coef": model.coef_,
        "intercept": model.intercept_,
    }

for name, res in results.items():
    print(name, "MSE:", res["mse_val"], "R2:", res["r2_val"])

model_energy_only MSE: 469.75563947142945 R2: 0.9841203903115117
model_hbond_env MSE: 3403.020364969947 R2: 0.884964371649685
model_structural MSE: 983.7563672967879 R2: 0.9667451205933031
model_all_selected MSE: 355.0367848393607 R2: 0.9879983440440469
model_energy_and_structural MSE: 387.4610813900624 R2: 0.9869022738101093


## Randomized Feature list:

In [14]:
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

target_col = "total_score"

all_features = ['dslf_fa13', 'fa_atr', 'fa_dun',
    'fa_elec', 'fa_intra_rep', 'fa_intra_sol_xover4', 'fa_rep', 'fa_sol',
    'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb', 'lk_ball_wtd',
    'omega', 'p_aa_pp', 'pro_close', 'rama_prepro', 'ref', 'yhh_planarity',
    'filter_total_score', 'buried_all', 'buried_np', 'contact_all',
    'contact_buried_core', 'contact_buried_core_boundary', 'degree',
    'degree_core', 'degree_core_boundary', 'exposed_hydrophobics',
    'exposed_np_AFIMLWVY', 'exposed_polars', 'exposed_total',
    'one_core_each', 'pack', 'res_count_all', 'res_count_buried_core',
    'res_count_buried_core_boundary', 'res_count_buried_np_core',
    'res_count_buried_np_core_boundary', 'ss_contributes_core', 'ss_mis',
    'total_hydrophobic', 'total_hydrophobic_AFILMVWY', 'total_sasa',
    'two_core_each', 'unsat_hbond', 'centroid_total_score', 'cbeta',
    'cenpack', 'env', 'hs_pair', 'linear_chainbreak', 'overlap_chainbreak',
    'pair', 'rg', 'rsigma', 'sheet', 'ss_pair', 'vdw']


def eval_model(feature_list):
    X_train = train_df[feature_list]
    y_train = train_df[target_col]

    X_val = val_df[feature_list]
    y_val = val_df[target_col]

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    return mse, r2

n_models = 500
min_size = 3
max_size = 15
results = []
known_combinations = []
for i in range(n_models):
    k = random.randint(min_size, max_size)
    feats = random.sample(all_features, k)
    feats = sorted(feats)
    if feats not in known_combinations:
        known_combinations.append(feats)
    else:
        print(i, "was skipped due to duplication.")
        continue
    mse, r2 = eval_model(feats)
    results.append({
        "iteration": i,
        "features": feats,
        "mse": mse,
        "r2": r2,
    })
    print("Iteration", i, "complete.")

# sort for best model, print 10 best performing
results_sorted = sorted(results, key=lambda d: (-d["r2"], d["mse"]))

for r in results_sorted[:10]:
    print(
        f"{r['iteration']:03d} | k={len(r['features'])} "
        f"| R2={r['r2']:.4f} | MSE={r['mse']:.2f} "
        f"| feats={r['features']}"
    )


Iteration 0 complete.
Iteration 1 complete.
Iteration 2 complete.
Iteration 3 complete.
Iteration 4 complete.
Iteration 5 complete.
Iteration 6 complete.
Iteration 7 complete.
Iteration 8 complete.
Iteration 9 complete.
Iteration 10 complete.
Iteration 11 complete.
Iteration 12 complete.
Iteration 13 complete.
Iteration 14 complete.
Iteration 15 complete.
Iteration 16 complete.
Iteration 17 complete.
Iteration 18 complete.
Iteration 19 complete.
Iteration 20 complete.
Iteration 21 complete.
Iteration 22 complete.
Iteration 23 complete.
Iteration 24 complete.
Iteration 25 complete.
Iteration 26 complete.
Iteration 27 complete.
Iteration 28 complete.
Iteration 29 complete.
Iteration 30 complete.
Iteration 31 complete.
Iteration 32 complete.
Iteration 33 complete.
Iteration 34 complete.
Iteration 35 complete.
Iteration 36 complete.
Iteration 37 complete.
Iteration 38 complete.
Iteration 39 complete.
Iteration 40 complete.
Iteration 41 complete.
Iteration 42 complete.
Iteration 43 complete